In [4]:
using DWave, JLD, Plots
#@pyimport dwave_sapi2.util as util
#@pyimport dwave_sapi2.embedding as embedding

In [5]:
#Future version of these ended up in DWave/src/QAC.jl


#produces and h vector and J matrix of max size but only with nonzeros on the first numBitsUsed bits
# h = 0
# Jij belongs to {-1, -5/6, ..., 5/6, 1}
function genInstance(numBitsUsed, numBitsTotal, nbrs, holes) 

    J = zeros(numBitsTotal, numBitsTotal);
    couplings = -1:1/6:1;
    idx = 1:length(couplings);
    
    for i = 1:numBitsUsed
        (i - 1) in holes && continue
        ns = nbrs[i - 1];
        for j = 1:length(ns)
            if i > ns[j] + 1
                J[Int(ns[j]) + 1, i] = couplings[rand(idx)];
            end
        end
    end
    
    
    return ProblemInstance(zeros(numBitsTotal), SparseMatrixCSC(J))
end

#Will generate the Pudenz Code according to a physical graph
function generatePudenzCode(solver = 0)
    
    #If no solver is given assume DW2X, otherwise get the properties via the DWave sapi
    if solver == 0
        props = getSolver()[:properties];
    elseif typeof(solver) == String
        props = getSolver(solver)[:properties];    
    else
        props = solver.properties;
    end

    #Get the relevant physical properties of the given solver
    workingQubits = props["qubits"];
    workingCouplers = props["couplers"];
    numPhysQubits = props["num_qubits"];
    physHoles = setdiff(0:1:(numPhysQubits - 1), workingQubits); #maybe workingQubits transpose

    #Get the relevant logical properties of the given solver
    numLogQubits = numPhysQubits/4;
    cellSize = sqrt(numPhysQubits/2);
    code = Dict();
    logNbrs = Dict();
    logHoles = [];


    #Create the code that converts the physical solver into a logical one via the Pudenz code
    for physBit = 0:8:(numPhysQubits - 8)

        #Upper 3 physical bits on each side will become a logical one
        leftLog = 2*floor(physBit/8); leftPhys = [physBit, (physBit + 1), (physBit + 2)];
        rightLog = leftLog + 1; rightPhys = [(physBit + 4), (physBit + 5), (physBit + 6)];

        if ~isempty(intersect(leftPhys, physHoles))
            leftPhys = []
            push!(logHoles, leftLog)
        end

        if ~isempty(intersect(rightPhys, physHoles))
            rightPhys = []
            push!(logHoles, rightLog)
        end
        

        #The lower bit on each side will become a penalty bit,
        # if present and coupled appropriately
        leftPen = physBit + 7; #penalty for left side sits on the right and vice versa
        if ~(leftPen in physHoles) && ~isempty(leftPhys)
            if isValidCoupling(leftPhys[1], leftPen, workingCouplers) && 
                isValidCoupling(leftPhys[2], leftPen, workingCouplers) &&
                isValidCoupling(leftPhys[3], leftPen, workingCouplers)
           
                push!(leftPhys, leftPen)
            end
        end

        rightPen = physBit + 3;
        if ~(rightPen in physHoles) && ~isempty(rightPhys)
            if isValidCoupling(rightPhys[1], rightPen, workingCouplers) &&
                isValidCoupling(rightPhys[2], rightPen, workingCouplers) &&
                isValidCoupling(rightPhys[3], rightPen, workingCouplers)

                push!(rightPhys, rightPen)
            end
        end

        code[leftLog] = leftPhys;
        code[rightLog] = rightPhys;
    end

    validLogQubits = setdiff(keys(code),logHoles);

    #Find the connectivity of the logical graph based on the code created above
    for (logBit, physBits) in code

        logNbrs[logBit] = [];
        #First check if this bit can even be used
        if logBit in logHoles
            continue;
        end


        if logBit % 2 == 0
            nbrs = intersect([(logBit + 1) (logBit + 2*cellSize) (logBit - 2*cellSize)], validLogQubits);
        else
            nbrs = intersect([(logBit - 1) (logBit - 2) (logBit + 2)], validLogQubits);
        end

        for bit in nbrs
            if ~(bit in logHoles)
                physNbrs = code[bit];
                if isValidCoupling(physBits[1], physNbrs[1], workingCouplers) &&
                    isValidCoupling(physBits[2], physNbrs[2], workingCouplers) &&
                    isValidCoupling(physBits[3], physNbrs[3], workingCouplers)
                    
                    push!(logNbrs[logBit], bit)
                end
            end
        end
    end
    dd
    save("code.jld", "code", code);
    save("logHoles.jld", "logHoles", logHoles);
    save("logNbrs.jld", "logNbrs", logNbrs);
    
    #return [code, holes, logNbrs]
end


function fractionPenaltyBits(code = load("code.jld", "code"))
    
    num = 0.0
    for ps in values(code)
        length(ps) == 4 && (num = num + 1)
    end
    
    return num/length(code)
end



function getSolver(solverName = "DW2X", url = "https://usci.qcc.isi.edu/sapi", token = "RnD-18901d8deba611aa9e02cb5556c783ea2597045d")
    return DWave.dwrm.RemoteConnection(url, token)[:get_solver](solverName);
end

#Assuming wcs = [a b; c d; ...]
function isValidCoupling(bit1, bit2, wcs)
    for i = 1:size(wcs, 1)
        ([bit1, bit2] == wcs[i,:] || [bit2, bit1] == wcs[i, :]) && return true
    end
    
    return false
end

isValidCoupling (generic function with 1 method)

In [ ]:
#Future versions of these ended up in DWave/src/embeddings.jl

#Take a logical ising problem defined by an h vector and J matrix and turn into a physical problem
#with beta as a the penalty stabilizer strength
function log2PhysHam(logH, logJ, beta, params = "")
    
    #Retrieve these from the code above
    code = load("code.jld");
    logNbrs = load("logNbrs.jld");
    numLogBits = length(logH);
    numPhysBits = numLogBits*4; #assuming Pudenz Code
    
    physH = zeros(numPhysBits, 1);
    physJ = zeros(numPhysBits, numPhysBits);
    
    logJ = sparse(logJ + logJ'); #since we will treat J as upper triangular and only care about nonzero elements
    
    for logBit = 1:numLogBits
        physBits = code(logBit - 1);
        
        #Create physical h for the bits we care about/can use
        if isempty(physBits) && logH[logBit] ~= 0
            throw(string("Logical qubit ", logBit, " is not valid, make sure to only include nonzero terms in the Hamiltonian for valid qubits"));
        end
        physH[physBits + 1] = logH[logBit]/2; #WHY DIVIDE BY 2??
        
        #Create physical J for the bits we care about/can use
        for logBit2 = (logBit + 1):numLogBits
            if ~(logBit2 in logNbrs[logBit])
                throw(string("Logical qubits ", logBit, " and ", logBit2, " are not coupled, make sure to only include nonzero terms in the Hamiltonian for valid qubits"));
            end
            
            physBits2 = code(logBit2 - 1);
            
            physJ[physBits[1], physBits2[1]] = logJ[logBit, logBit2];
            physJ[physBits[2], physBits2[2]] = logJ[logBit, logBit2];
            physJ[physBits[3], physBits2[3]] = logJ[logBit, logBit2];
        end
        
        #Create stabilizer terms for penalty bits we have for the logical bits we care about
        if logH[logBit] ~= 0 || sum(logJ[:, logBit]) > 0
            if length(physBits) == 4
                physJ[physBits[1], physBits[4]] = -beta; 
                physJ[physBits[2], physBits[4]] = -beta;
                physJ[physBits[3], physBits[4]] = -beta;
            end
        end
    end
    
    
    return ProblemInstance(physH, sparse(physJ), params)
end

#Take the solutions given by DWave and convert them back to a logical solution
#strategy of majority vote will pick what most phys bits got, ep/other will only accept unanimous vote or otherwise 0 (tie)
function decodeSolutions(sols, strat = "majvote")
    
    numLogBits = size(sols[1])/4;
    numReads = length(sols);
    holes = load("holes.jld"); #get this from the code
    code = load("code.jld");
    
    decodedSols = 3*ones(numLogsBits, numReads); #why *3...?
    
    for logBit = setdiff(0:(numLogBits - 1), holes)
        physBits = code(logBit) + 1;
        majVote = sum(sols[:, physBits[1:3]], 2); #definitely check that this is right...
        #why do this??
        if majVote[1] == 12
            continue;
        end
        
        if strat == "majvote"
            decodedSols[logBit + 1, :] = 1*(majVote .> 0) - 1*(majVote .< 0)
        else
            decodedSols[logBit + 1, :] = 1*(majVote .== 3) - 1*(majVote .== -3)
        end
    end
    
    return decodedSols;
end

In [ ]:
#This was rendered unnecessary since run_experiment exists in DWave/src/experiments.jl

#Take the number of instances to be produced and the fraction of (logical) bits to be used
#Generate these instances and send them to DWave
function runQAC(numInstances = 1, fractionBitsUsed = 1, beta = 1)
    
    code = load("code.jld");
    logNbrs = load("logNbrs");
    numBitsTotal = length(code);
    numBitsUsed = fractionBitsUsed*numBitsTotal;
    
    file = jldopen("data.jld", "w");
    
    for i = 1:numInstances
        logH, logJ = genInstance(numBitsUsed, numBitsTotal, logNbrs);
        submitProblem(log2PhysHam(logH, logJ, beta));
        
        #Maybe not necessary to save these?
        file[i] = [logH, logJ]; #can always reproduce the physical h and J's using log2PhysHam
    end
    
    close(file);
end

function init(solver = 0)
    generatePudenzCode(solver);
end

In [8]:
# Generating and Solving Instances
#----------------------------------
dir = "test/"

problem_size = 288 #1152/4 = 228
solver_size = 1152 #DW2X = 1152
num_insts = 1 #1000
#num_embeddings = 18
num_per_call = 15
num_samples_dw = 1000
num_samples_hfs = num_samples_dw*10
num_runs_dw = 20
num_runs_hfs = num_runs_dw
betas = [0.1, 0.2, 0.3, 0.4, 0.5]
num_bs = length(betas)

successQAC = zeros(num_insts, 4)
successC = zeros(num_insts, 3)
num_posterior_samples = 1000

eg = EmbeddingGenerator([GaugeEmbedding=>solver_size])
hfs_eg = EmbeddingGenerator([GaugeEmbedding=>problem_size])
props = getSolver()[:properties]
props[:token] = "RnD-18901d8deba611aa9e02cb5556c783ea2597045d"
nbrs = load("logNbrs.jld", "logNbrs")
holes = load("logHoles.jld", "logHoles")

println("Starting")
for i = 1:num_insts
    inst = genInstance(problem_size, solver_size/4, nbrs, holes)
    
    # HFS for finding GS Energy
    #---------------------------
    println("Running HFS")
    exper = ExperimentDescription(instance=inst, R=num_samples_hfs, name=string("instHFS_", i),solver=:HFS, solver_properties=props, 
    embedding_generator=hfs_eg, decode_opts=[1])
    (_, es, _) = run_experiment(exper, minruns=1,maxruns=num_runs_hfs,vc=0.2,vcmode=:prob, N=1,
        savesols=false,save_encoded=false,tosave=false, save_embeddings=false,coupleruns=false,timeout=2.,verbose=true)
    
    gs_e = minimum(es)
    
    
    # Pudenz Code
    #-------------
    tmpVals = zeros(3)
    for b in betas
        # Create the physical problem for the given beta
        p_emb = PudenzEmbedding(b)
        inst_p = applyEmbedding(p_emb, inst)
        
        # Send pudenz/QAC problem off to DWave
        println("Running Pudenz Code on DWave with beta = ", b)
        exper = ExperimentDescription(instance=inst_p, R=num_samples_dw, name=string("instQAC_", i),solver=:DWave,
        solver_properties=props,embedding_generator=eg, decode_opts=[1])
        (_, es_dw, _) = run_experiment(exper, minruns=num_runs_dw,maxruns=num_runs_dw,vc=0.2,vcmode=:prob, N=num_per_call,
                savesols=true,save_encoded=false,tosave=true, 
                save_embeddings=false,coupleruns=false,savepath=dir,timeout=2.,verbose=true)
        
        # Select the best results
        ps = get_posteriors(es_dw,false,num_posterior_samples,true_Egs=gs_e)[:,end]
        if mean(ps) > tmpVals[1]
            tempVals = [mean(ps) std(ps) b]
        end
    end
    
    successQAC[i] = [tempVals[1:2], gs_e, tempVals[3]]
    
    
    # C/Repetition Code
    #-------------------
    # NOTE: pudenz code with beta = 0 == 3C repetition?
    p_emb = PudenzEmbedding(0)
    inst_c = applyEmbedding(p_emb, inst)
    exper = ExperimentDescription(instance=inst_c, R=num_samples_dw, name=string("instC_", i), solver=:DWave,
    solver_properties=props,embedding_generator=eg, decode_opts=[1])
    (_, es_dw, _) = run_experiment(exper, minruns=num_runs_dw,maxruns=num_runs_dw,vc=0.2,vcmode=:prob, N=num_per_call,
            savesols=true,save_encoded=false,tosave=true, 
            save_embeddings=false,coupleruns=false,savepath=dir,timeout=2.,verbose=true)
    
    #gives prob of success for 4C given prob of success for 3C (assuming negligible correlations)
    ps = 1 - (1 - get_posteriors(es_dw,false,num_posterior_samples,true_Egs=gs_e)[:,end]).^(4/3)
    successC[i] = [mean(ps), std(ps), gs_e]
end

println("Done")

#Can all this just be gotten from what's already stored?
save(string(dir,"success_QAC.jld"), "mean", successQAC[:, 1])
save(string(dir,"success_QAC.jld"), "std", successQAC[:, 2])
save(string(dir,"success_QAC.jld"), "gs_e", successQAC[:, 3])
save(string(dir,"success_QAC.jld"), "betas", successQAC[:, 4])

save(string(dir,"success_C.jld"), "mean", successC[:, 1])
save(string(dir,"success_C.jld"), "std", successC[:, 2])
save(string(dir,"success_C.jld"), "gs_e", successC[:, 3])


# Results
#---------
plot(successQAC[:, 1], successC[:, 1]) #need to add color depending on beta value... pyplot, gadfly, etc.

LoadError: [91mInterruptException:[39m

In [ ]:
#desc = load("test_instance/ExperimentDescriptions.jld2", "ExperimentDescriptions")[1]
one = load("test_instance/1.jld2")
des = one["decoded_energies"]

#histogram(one["decoded_energies"])

println(get_posterior_variations(des,false,1000,vcmode=:Prob)[1])
println(get_posteriors(des,false,1000,true_Egs=false)[:,end])
println(minimum(des))

#histogram(des)

In [ ]:
nbrs = load("logNbrs.jld", "logNbrs")
holes = load("logHoles.jld", "logHoles")
inst = genInstance(288, 288, nbrs, holes)

p_emb = PudenzEmbedding(0.1)
inst_p = applyEmbedding(p_emb, inst)

exper = ExperimentDescription(instance=inst_p, R=num_samples_dw, name=string("instQAC_"),
        solver=:DWave, solver_properties=props,embedding_generator=eg, decode_opts=[1])

#dw_adj = SparseMatrixCSC(zeros(1152, 1152))
#adj_set = util.get_hardware_adjacency(getSolver())

#for a in adj_set
#    dw_adj[a[1] + 1, a[2] + 1] = 1 
#end

#emb = SAPIEmbedding(spones(inst.J), dw_adj)

#emb = SAPIEmbedding(1152, 0)

In [10]:
load("test_instance/1.jld2")

Dict{String,Any} with 6 entries:
  "decoded_solutions"     => Bool[false false … false false; false false … fals…
  "decoded_energies"      => [-70.1667; -81.0; … ; -73.6667; -78.8333]
  "sweeps"                => [20; 20; … ; 20; 16]
  "energies"              => [-367.333; -444.0; … ; -434.667; -457.667]
  "date"                  => 2018-04-22T12:26:17.979
  "ExperimentDescription" => DWave.ExperimentDescription(ProblemInstance(h,J,pr…

In [13]:
x = [1 2 3]
(1 - x).^2

1×3 Array{Int64,2}:
 0  1  4